In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
from code.python.mpmath_integration import quad_phi
from mpmath import mp

from scipy import stats

In [159]:
x = mp.mpf('1/2')
alpha = mp.mpf('13')
beta = mp.mpf('7/2')
mu = mp.mpf('1/4')
delta = mp.mpf('1')

xmu = x - mu
gamma = mp.sqrt(alpha ** 2 - beta ** 2)
omega = mp.sqrt(xmu ** 2 + delta ** 2)

In [160]:
mp_result = quad_phi(x, alpha, beta, mu, delta, digits=100)
mp_result

mpf('0.4733304299905477472382706779792114831944708059541335904268965804037482362449676285016570392186719003154')

In [161]:
cache = {}

def Ik(k):
    s = 0
    for j in range(2*k + 1 + 1):
        m = k + 1 - j
        if abs(m) not in cache:
            bm = mp.besselk(m, alpha * omega)
            cache[m] = bm
        else:
            bm = cache[abs(m)]
            
        s += mp.binomial(2*k + 1, j) * (-1) ** (-j) * (omega * beta / alpha / xmu) ** (j) * bm

    return 2 * s

In [162]:
cache

{}

In [163]:
%%time
mp.dps = 100

C = delta * mp.exp(delta * gamma + xmu * beta) / (2 * mp.pi)

N = 50
s = 0

z = xmu ** 2 * alpha / omega

for k in range(N):
    r = z ** k / mp.fac2(2*k + 1)
    # q = mp.quad(lambda t: (t - xmu / beta)**(2*k + 1) * t**(-k - 2) * mp.exp(-omega ** 2 / 2 / t - alpha ** 2 / 2 * t), [0, mp.inf])
    q = Ik(k)
    t = r * q
    s += t
    print(k, float(r), float(q), float(t))

mp.mpf('1/2') + C * s * xmu * alpha / omega

0 1.0 -7.546641030270782e-08 -7.546641030270782e-08
1 0.26274692712269404 -1.9387692503075078e-08 -5.09405662918267e-09
2 0.04142156862745098 -8.316020791840879e-09 -3.444626259365462e-10
3 0.004664309945770514 -5.002784394022533e-09 -2.3334537005584817e-11
4 0.00040851103513300747 -3.876459911578666e-09 -1.5835766511306076e-12
5 2.9273187048247804e-05 -3.677819168150296e-09 -1.0766148843889477e-13
6 1.774947679388067e-06 -4.131153753759348e-09 -7.332581768430457e-15
7 9.327240971255428e-08 -5.363711045464483e-09 -5.002862542123161e-16
8 4.324771594818099e-09 -7.906216991979375e-09 -3.4192582669380594e-17
9 1.7941901795462659e-10 -1.3047048793040845e-08 -2.3408886816534845e-18
10 6.7345422335642296e-12 -2.383629344635491e-08 -1.605265249061074e-19
11 2.308017753191747e-13 -4.777171795647351e-08 -1.1025797314400984e-20
12 7.277094868749072e-15 -1.0422959112785498e-07 -7.584886227683273e-22
13 2.1244825723823813e-16 -2.4597368624531276e-07 -5.2256680969281883e-23
14 5.774495871922626e-18

mpf('0.473330429990547747238270677979211483194470805954133590426900284256572628288071842399609346487842670949')